# Phase 3: Advanced Analytics - RFM Customer Segmentation

**Team:** Code Serpents

**Team Member:** G.A.Dilsara Thiranjaya

---

## Executive Summary

This notebook implements a comprehensive RFM (Recency, Frequency, Monetary) analysis to segment customers of "Unique Gifts Ltd." based on their purchasing behavior. The analysis will help identify high-value customers, at-risk segments, and opportunities for targeted marketing strategies.

## Objectives

1. **Calculate RFM Metrics**: Compute Recency, Frequency, and Monetary values for each customer
2. **Assign RFM Scores**: Use quintile-based scoring (1-5) for each RFM dimension
3. **Customer Segmentation**: Map RFM scores to descriptive business segments
4. **Business Insights**: Generate actionable recommendations for each segment
5. **Wholesaler Analysis**: Investigate the hypothesis of two distinct customer groups

---

## 1. Setup and Data Loading

In [1]:
# Import required libraries
import sys
import os

# Add the src directory to Python path
sys.path.append('../src')

# Import our custom RFM analyzer
from rfm_segmentation import RFMAnalyzer, load_and_validate_data

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None) # Display all columns of the DataFrame without truncation
pd.set_option('display.max_rows', 100) # Display a maximum of 100 rows in the DataFrame

print("📦 All libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load the cleaned data
DATA_PATH = "../data/online_retail_clean.csv"

data = load_and_validate_data(DATA_PATH)

# Display basic information about the dataset
print("\n📊 Dataset Overview:")
print(f"Shape: {data.shape}")
print(f"Date Range: {data['InvoiceDate'].min()} to {data['InvoiceDate'].max()}")
print(f"Unique Customers: {data['Customer ID'].nunique():,}")
print(f"Total Transactions: {data['Invoice'].nunique():,}")
print(f"Total Revenue: £{data['TotalPrice'].sum():,.2f}")

# Display first few rows
print("\n🔍 Sample Data:")
display(data.head())

📦 All libraries imported successfully!
📅 Analysis Date: 2025-08-19 09:30:49
✅ Data loaded successfully: 13 rows, 13 columns

📊 Dataset Overview:
Shape: (13, 13)
Date Range: 2009-12-01 07:45:00 to 2009-12-01 09:06:00
Unique Customers: 2
Total Transactions: 3
Total Revenue: £710.60

🔍 Sample Data:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,Year,Month,DayOfWeek,HourOfDay
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,2009,12,1,7
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,12,1,7
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,12,1,7
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,2009,12,1,7
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,2009,12,1,7


## 2. RFM Analysis Implementation

### 2.1 Initialize RFM Analyzer

In [2]:
# Initialize RFM analyzer
rfm_analyzer = RFMAnalyzer(data)

print("🚀 RFM Analyzer initialized successfully!")
print("Ready to perform comprehensive customer segmentation analysis.")

Data validation passed. All required columns are present.
🚀 RFM Analyzer initialized successfully!
Ready to perform comprehensive customer segmentation analysis.


### 2.2 Run Complete RFM Analysis

In [3]:
# Execute the complete RFM analysis workflow
results = rfm_analyzer.run_complete_analysis()

# Extract results for easy access
rfm_data = results['rfm_data']
rfm_scored = results['rfm_scored']
rfm_final = results['rfm_final']
segment_summary = results['segment_summary']
summary_stats = results['summary_stats']

print("\n✅ RFM Analysis completed successfully!")

🔄 Starting RFM Analysis...
📊 Calculating RFM metrics...
📈 Generating summary statistics...
🏷️ Assigning RFM scores...


ValueError: Bin labels must be one fewer than the number of bin edges

## 3. RFM Metrics Analysis

### 3.1 Basic RFM Statistics

### 3.2 RFM Scores and Segments

## 4. Customer Segment Analysis

### 4.1 Segment Summary Statistics

### 4.2 Detailed Segment Characteristics

## 5. Data Visualizations

### 5.1 Generate All Visualizations

### 5.2 RFM Distributions Analysis

**Analysis of RFM Distributions:**

- **Recency Distribution**: Shows the distribution of days since last purchase. A right-skewed distribution indicates most customers purchased recently, with a tail of customers who haven't purchased in a long time.

- **Frequency Distribution**: Displays the number of transactions per customer. Typically right-skewed, showing most customers make few purchases, while a small group makes many purchases.

- **Monetary Distribution (Log Scale)**: Shows customer spending patterns. The log scale helps visualize the wide range of spending behaviors and potential bimodal distribution suggesting retail vs. wholesale customers.

- **Box Plot**: The monetary box plot is crucial for investigating the wholesaler hypothesis - extreme outliers may indicate wholesale customers.

### 5.3 Customer Segment Analysis

**Segment Analysis Insights:**

- **Customer Count by Segment**: Shows the relative size of each customer segment
- **Revenue Distribution**: Reveals which segments contribute most to total revenue
- **Average Monetary Value**: Identifies the highest-spending segments
- **Customer Distribution**: Provides overview of segment composition

## 6. Export Results

### 7.1 Export to Excel and Save charts

### 7.2 Save Key DataFrames for Integration